### Processing

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
path = "/content/drive/My Drive/M2/DR"

import sys
sys.path.append(path)

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from scipy import io
from math import sqrt

dataset = "MNIST"
dataset = "USPS"

#################################################
# Dataset
#################################################

# MNIST
if dataset == "MNIST":
    from keras.datasets import mnist
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

# USPS
if dataset == "USPS":
    mat = io.loadmat(path+"/USPS.mat")
    X_train = X_test = mat['X'].reshape((mat['X'].shape[0], int(sqrt(mat['X'].shape[1])), int(sqrt(mat['X'].shape[1]))))
    y_train = y_test = mat['y']

#####################
# Pre processing
#####################

X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

print(X_train.shape)
print(X_test.shape)

min_X = min(X_train.min(), X_test.min())
max_X = max(X_train.max(), X_test.max())
X_train, X_test = (X_train - min_X) / (max_X - min_X), (X_test - min_X) / (max_X - min_X)


#################################################
# Functions
#################################################
num_examples_to_generate=16
def plot_images(predictions, epoch):
  fig = plt.figure(figsize=(4,4))

  for i in range(predictions.shape[0]):
      plt.subplot(4, 4, i+1)
      plt.imshow(predictions[i, :, :, 0] * 255.0, cmap='gray')
      #plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
      plt.axis('off')

  #plt.savefig('image_at_epoch_{:04d}.png'.format(epoch))
  plt.show()

def generate_and_save_images(model, epoch, test_input):
  predictions = model(test_input, training=False)

  plot_images(predictions, epoch)

In [0]:
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

nmi = normalized_mutual_info_score
ari = adjusted_rand_score

kmeans = KMeans(n_clusters=10, n_init=20)
kmeans.fit(X_train.reshape((60000, 28*28)))

print(nmi(kmeans.labels_, y_train))
print(ari(kmeans.labels_, y_train))

## **1. Deep Autoencoder**

### Model

In [0]:
from tensorflow import keras
from tensorflow.keras import Input, layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score

nmi = normalized_mutual_info_score
ari = adjusted_rand_score



#################################################
# Hyper-parameters
#################################################
epoch = 100
batch_size = 256

encoding_dim = 7*7
hidden_dim = 512

#activation='elu'
activation='leaky_relu'
optimizer='adam'
#loss='mean_squared_error'
loss='binary_crossentropy'
#################################################

out_activation = 'sigmoid' if loss=='binary_crossentropy' else 'linear'
activation = None if activation=='leaky_relu' else activation

def make_autoencoder_model():
    #################################################
    # Auto-encoder
    #################################################
    input_img = Input(shape=(784,))


    ####### Encoder #######
    #   Layer
    encoded = Dense(hidden_dim, activation=activation)(input_img)
    if activation is None: encoded = layers.LeakyReLU()(encoded)

    #   Layer
    encoded = Dense(hidden_dim//4, activation=activation)(encoded)
    if activation is None: encoded = layers.LeakyReLU()(encoded)

    #   Layer
    encoded = Dense(encoding_dim, activation=out_activation)(encoded)


    ####### Decoder #######
    #   Layer
    decoded = Dense(hidden_dim//4, activation=activation)(encoded)
    if activation is None: decoded = layers.LeakyReLU()(decoded)

    #   Layer
    decoded = Dense(hidden_dim, activation=activation)(decoded)
    if activation is None: decoded = layers.LeakyReLU()(decoded)

    #   Layer
    decoded = Dense(784, activation=out_activation)(decoded)


    ####### Make model #######
    autoencoder = Model(input_img, decoded)

    return autoencoder

### Training

In [0]:

autoencoder = make_autoencoder_model()
autoencoder.compile(optimizer=optimizer, loss=loss)
autoencoder.summary()
####### Train #######
autoencoder.fit(X_train, X_train,
                epochs=epoch,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(X_test, X_test))


####### Encode images #######
encoder = Model(input_img, encoded)
encoded_images = encoder.predict(X_train)
#################################################



#################################################
# KMeans
#################################################
print("run KMeans")
kmeans = KMeans(n_clusters=10, n_init=20)
kmeans.fit(encoded_images)

print(nmi(kmeans.labels_, y_train))
print(ari(kmeans.labels_, y_train))
#################################################

## **2. Deep Convolutional Autoencoder**

### Model

In [0]:
from tensorflow import keras
from tensorflow.keras import Input, layers, models
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score, adjusted_rand_score
from math import sqrt

nmi = normalized_mutual_info_score
ari = adjusted_rand_score



#################################################
# Hyper-parameters
#################################################
epoch = 10
batch_size = 256

#activation='elu'
activation='leaky_relu'
optimizer='adam'
#loss='mean_squared_error'
loss='binary_crossentropy'

use_batch_norm_encoded = False
use_batch_norm_decoded = False
#################################################

out_activation = 'sigmoid' if loss=='binary_crossentropy' else 'linear'
activation = None if activation=='leaky_relu' else activation

def make_conv_autoencoder_model(img_height=28, img_width=28):
  big_img = img_height >= 18 and img_width >= 18
  divisor = 4 if big_img else 2
  encoding_height, encoding_width = img_height//divisor, img_width//divisor
  encoding_dim = encoding_height * encoding_width

  #################################################
  # Convolutional Auto-encoder
  #################################################
  input_img = Input((img_height, img_width, 1))

  ####### Encoder #######
  #   Layer
  encoded = layers.Conv2D(32, (3, 3), activation=activation)(input_img)
  if use_batch_norm_encoded: encoded = layers.BatchNormalization()(encoded)
  if activation is None:     encoded = layers.LeakyReLU()(encoded)
  encoded = layers.MaxPooling2D((2, 2))(encoded)

  #   Layer
  encoded = layers.Conv2D(64, (3, 3), activation=activation)(encoded)
  if use_batch_norm_encoded: encoded = layers.BatchNormalization()(encoded)
  if activation is None:     encoded = layers.LeakyReLU()(encoded)
  if big_img:                encoded = layers.MaxPooling2D((2, 2))(encoded) # otherwise we will have dimension issues

  #   Layer
  encoded = layers.Conv2D(64, (3, 3), activation=activation)(encoded)
  if use_batch_norm_encoded: encoded = layers.BatchNormalization()(encoded)                          
  if activation is None:     encoded = layers.LeakyReLU()(encoded)

  #   Layer
  encoded = layers.Flatten()(encoded)
  encoded = layers.Dense(encoding_dim, activation=out_activation)(encoded)


  ####### Decoder #######
  decoded_input = encoded if encoding_dim == encoding_height * encoding_width else layers.Dense(encoding_height * encoding_width)(encoded)
  decoded = layers.Reshape((encoding_height, encoding_width, 1))(decoded_input)

  #   Layer
  strides = 2 if big_img else 1
  decoded = layers.Conv2DTranspose(64,(3, 3), strides=strides, activation=activation, padding='same')(decoded)
  if use_batch_norm_decoded: decoded = layers.BatchNormalization()(decoded)
  if activation is None:     decoded = layers.LeakyReLU()(decoded)

  #   Layer
  decoded = layers.Conv2DTranspose(64,(3, 3), strides=2, activation=activation, padding='same')(decoded)
  if use_batch_norm_decoded: decoded = layers.BatchNormalization()(decoded)
  if activation is None:     decoded = layers.LeakyReLU()(decoded)

  #   Layer
  decoded = layers.Conv2DTranspose(32,(3, 3), activation=activation, padding='same')(decoded)
  if use_batch_norm_decoded: decoded = layers.BatchNormalization()(decoded)
  if activation is None:     decoded = layers.LeakyReLU()(decoded)

  #   Layer
  decoded = layers.Conv2D(1, (3, 3), activation=out_activation, padding='same')(decoded)


  ####### Make model #######
  autoencoder = models.Model(input_img, decoded)
  encoder = models.Model(input_img, encoded)
  #decoder = models.Model(encoded, decoded)

  return autoencoder, encoder, decoded

### Training

In [0]:
epoch = 100

import os
import tensorflow as tf

autoencoder, encoder, decoded = make_conv_autoencoder_model(X_train.shape[1], X_train.shape[2])
autoencoder.compile(optimizer=optimizer, loss=loss)

autoencoder.summary()

"""ae_checkpoint_dir = path+'/ae_training_checkpoints'
ae_checkpoint_prefix = os.path.join(ae_checkpoint_dir, "ckpt")
ae_checkpoint = tf.train.Checkpoint(autoencoder=autoencoder)

ae_checkpoint.restore(tf.train.latest_checkpoint(ae_checkpoint_dir))"""

"""for _ in range(8):
  generate_and_save_images(autoencoder, 0, 
                        test_input=X_train[np.random.randint(low=0,high=train_images.shape[0],size=16)])"""

####### Train #######
autoencoder.fit(X_train, X_train,
                epochs=epoch,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(X_test, X_test))

#ae_checkpoint.save(file_prefix = ae_checkpoint_prefix)

for _ in range(8):
  generate_and_save_images(autoencoder, 0, 
                        test_input=X_train[np.random.randint(low=0,high=X_train.shape[0],size=16)])

#################################################



#################################################
# KMeans
#################################################
####### Encode images #######
encoded_images = encoder.predict(X_train)
print(encoded_images.shape)

####### KMeans #######
print("run KMeans")
kmeans = KMeans(n_clusters=10, n_init=20)
kmeans.fit(encoded_images)

print(nmi(kmeans.labels_, y_train.ravel()))
print(ari(kmeans.labels_, y_train.ravel()))
#################################################

## **3. LLE**



### Model

\begin{equation}
\frac{\partial \mathcal{L}}{\partial w_i} = 2G_i w_i - \lambda \mathbf{1} \\
\\
\frac{\partial \mathcal{L}}{\partial \lambda} = \mathbf{1}^T w_i - \mathbf{1}
\end{equation}

In [0]:
from time import time
import numpy as np
from scipy import sparse
from scipy.linalg import solve
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import LocallyLinearEmbedding



class LLE():
    def __init__(self, n_neighbors=5, n_components=2, n_jobs=None, verbose=True, 
                 learning_rate=0.0001, neighbors_update=False, method="direct",
                 reg=1e-3):
        self.n_neighbors = n_neighbors
        self.n_components = n_components
        self.n_jobs = n_jobs
        self.verbose = verbose        
        self.learning_rate = learning_rate        # Used only if method="gradient"
        self.neighbors_update = neighbors_update
        self.method = method
        self.reg = reg

        self.W = None                            # Sparse full Weight matrix [n_samples x n_samples]
        self.B = None                            # Reduced Weight matrix [n_samples x n_neighbors]
        self.lagrange_lambda = 0                 # Lambda (lagrange multiplier)
        self.knn = None                          # Store kNN result to avoid computation each time fit is called
        self.ind = None                          # Store kNN neighbors indexes

    def _update_W(self, regularization=True):
        n_samples = self.B.shape[0]
        indptr = np.arange(0, n_samples * self.n_neighbors + 1, self.n_neighbors)
        rowsum = self.B.sum(1)[:,None]
        rowsum[rowsum == 0] = 1 
        data = self.B 
        if regularization:
            data = data / rowsum
        self.W = sparse.csr_matrix((data.ravel(), self.ind.ravel(), indptr),
                      shape=(n_samples, n_samples))

    def get_W(self):
        self._update_W()
        return self.W

    def compute_weight_loss(self, X):
        W = self.get_W()
        X_csr = sparse.csr_matrix(X)
        #return np.square(X - np.dot(W.todense(), X)).sum()
        return (X_csr - W.dot(X_csr)).power(2).sum()

    def _one_fit(self, X):
        Z = X[self.ind]
        n_samples, n_neighbors = X.shape[0], Z.shape[1]
        ones = np.ones(n_neighbors)

        for i, A in enumerate(Z.transpose(0, 2, 1)):
            #C = A.T - X[i]  
            C = X[i] - A.T
            G = np.dot(C, C.T)

            if self.method == "gradient":
                # Gradient descent way
                w_gradient = 2 * np.dot(G, self.B[i, :]) - self.lagrange_lambda * ones
                lambda_gradient = np.dot(ones, self.B[i, :]) - 1

                self.lagrange_lambda -= self.learning_rate * lambda_gradient
                self.B[i, :] -= self.learning_rate * w_gradient

                b_sum = self.B[i, :].sum()
                if b_sum != 0: 
                    self.B[i, :] = self.B[i, :] / b_sum 

            elif self.method == "direct":
                # Direct way, from sklearn's "barycenter_weights" function
                # https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/manifold/_locally_linear.py
                trace = np.trace(G)
                if trace > 0:
                    R = self.reg * trace
                else:
                    R = self.reg
                G.flat[::Z.shape[1] + 1] += R
                w = solve(G, ones, sym_pos=True)
                self.B[i, :] = w / np.sum(w)


    def fit(self, X, epoch=1):
        # Run kNN algorithm      
        if self.knn is None or self.neighbors_update:
            if self.verbose: print("Run kNN...")
            self.knn = NearestNeighbors(self.n_neighbors + 1, n_jobs=self.n_jobs).fit(X)
            self.ind = self.knn.kneighbors(X, return_distance=False)[:, 1:]
        
        # Init B matrix
        if self.B is None:
            """self.B = np.random.random((X.shape[0], self.n_neighbors))
            self.B /= self.B.sum(1)[:,None]"""
            self.B = np.zeros((X.shape[0], self.n_neighbors))

        # Train
        if self.verbose: 
            print("Train...")
            #print("LLE loss = ", self.compute_weight_loss(X))
        for ep in range(epoch):
            t0 = time()
            self._one_fit(X)
            t1 = time()
            if self.verbose:
                print(": %.2g sec" % (t1 - t0))
            if self.method != "direct": 
                if ep % 100 == 0 and self.verbose:
                    print(ep, " \ ", epoch)
                    print("LLE loss = ", self.compute_weight_loss(X))
            else:
                break

        if self.verbose:
            print("LLE loss = ", self.compute_weight_loss(X))

        return self

    def transform(self, X):
        return None

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)


### Training

In [0]:
X = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2])
X = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2])
print(X.shape)

lle = LLE(method="gradient")
lle.fit(X, epoch=1000)

## **4. AE LLE**



### Model

In [0]:
import tensorflow as tf
from scipy import sparse

mse = tf.keras.losses.MeanSquaredError()

def ae_lle_loss(X_encoded, W):
    X_encoded_sparse = sparse.csr_matrix(X_encoded)
    return mse(X_encoded, W.dot(X_encoded_sparse).todense())

def make_ae_lle_optimizer():
    return tf.keras.optimizers.Adam()

### Training

In [0]:
epoch = 50

import os
import tensorflow as tf
from statistics import mean

autoencoder, encoder, decoded = make_conv_autoencoder_model(X_train.shape[1], X_train.shape[2])
autoencoder.compile(optimizer=optimizer, loss=loss)
ae_lle_optimizer = make_ae_lle_optimizer()

autoencoder.summary()

lle = LLE(neighbors_update=True, verbose=False) # we need to recompute neighbors at each iteration autoencoder encoding changes over time

# Train encoder according to the second term of the loss
def ae_lle_train_step(X, W):
    with tf.GradientTape() as ae_lle_tape:
      X_encoded = encoder(X, training=True)
      encoder_loss = ae_lle_loss(X_encoded, W)

    gradients_of_ae_lle = ae_lle_tape.gradient(encoder_loss, encoder.trainable_variables)

    ae_lle_optimizer.apply_gradients(zip(gradients_of_ae_lle, encoder.trainable_variables))

    return encoder_loss

####### Train #######
for ep in range(epoch):
    print("_________________________________________________________________")
    print("epoch : ",ep, " / ", epoch)
    #losses = {"Autoencoder":[], "LLE":[], "AE-LLE":[]}

    # Autoencoder Training
    #print("--- Autoencoder update...")
    autoencoder.fit(X_train, X_train,
                    epochs=1,
                    batch_size=batch_size,
                    shuffle=True, verbose=0,
                    validation_data=(X_test, X_test))
    results = autoencoder.evaluate(X_train, X_train, batch_size=batch_size, verbose=0)
    print("Autoencoder loss = ", results)
    
    # LLE Training
    #print("--- LLE update...")
    X_encoded = encoder.predict(X_train)
    #print(X_encoded.shape)
    lle.fit(X_encoded)
    print("LLE loss         = ", lle.compute_weight_loss(X_encoded))

    # Encoder Training
    #print("--- Encoder update...")
    encoder_losses = []
    W = lle.get_W()
    """for it in range(int(X_train.shape[0] / batch_size)):
        indexes = np.random.randint(low=0,high=X_train.shape[0],size=batch_size)
        X_batch = X_train[indexes]  
        W_batch = W[indexes]
        print(W_batch.shape)
        print(X_batch.shape)
        input()
        encoder_loss = ae_lle_train_step(X_batch, W_batch)
        encoder_losses.append(encoder_loss)
    print("Encoder loss = ", mean(encoder_losses))"""
    encoder_loss = ae_lle_train_step(X_train, W)
    print("Encoder loss     = ", encoder_loss.numpy())

for _ in range(8):
  generate_and_save_images(autoencoder, 0, 
                        test_input=X_train[np.random.randint(low=0,high=X_train.shape[0],size=16)])

#################################################






"""ae_checkpoint_dir = path+'/ae_training_checkpoints'
ae_checkpoint_prefix = os.path.join(ae_checkpoint_dir, "ckpt")
ae_checkpoint = tf.train.Checkpoint(autoencoder=autoencoder)

ae_checkpoint.restore(tf.train.latest_checkpoint(ae_checkpoint_dir))

ae_checkpoint.save(file_prefix = ae_checkpoint_prefix)"""

"""for _ in range(8):
  generate_and_save_images(autoencoder, 0, 
                        test_input=X_train[np.random.randint(low=0,high=train_images.shape[0],size=16)])"""

"""for it in range(int(X_train.shape[0] / batch_size)):
    X_batch = X_train[np.random.randint(low=0,high=X_train.shape[0],size=batch_size)]  
    autoencoder.fit(X_batch, X_batch,
                    epochs=1,
                    batch_size=batch_size,
                    shuffle=True, verbose=0)
    
results = autoencoder.evaluate(X_test, X_test, batch_size=batch_size)
print('Autoencoder validation:', results)"""